In [1]:
import json
import re
from tqdm import tqdm
from collections import Counter
import pandas as pd


In [2]:
tmp = []
def process_line(line):
    try:
        data = json.loads(line.strip())
        text = data.get('text', '')
        author_username = data.get('author', {}).get('username', '')
        author_uid = data.get('author', {}).get('id', '')
        author_nrv = data.get('author', {}).get('num_reviews', 0)
        offerID = data.get('offering_id', '')
        rating = data.get('ratings', {}).get('overall', '0')
        id = data.get('id', '')
        
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
    return author_uid, offerID, id, author_nrv, text, rating

In [3]:
file_path = './data/reviews/tripAdvisor.txt'  

lauthor = []
loid = []
lid = []
lnrv = []
ltext = []
lrate = []

with open(file_path, 'r', encoding='utf-8') as file:
  for line in tqdm(file):
    author, offerID, id, author_nrv, text, rate = process_line(line)
    lauthor.append(author)
    loid.append(offerID)
    lid.append(id)
    lnrv.append(author_nrv)
    ltext.append(text)
    lrate.append(rate)

lau = set(lauthor)
lo = set(loid)
li = set(lid)
print(len(lau), len(lo), len(li))

878561it [00:10, 82902.31it/s]


576689 3945 878561


In [4]:
tmp = Counter(lauthor)
finalAuth = []
for k, v in tmp.items():
  if v >= 5:
    finalAuth.append(k)
print(len(finalAuth))
if "" in finalAuth:
  finalAuth.remove("")
print(len(finalAuth))

11974
11973


In [19]:
# given lists lauthor, loid, lid, lnrv and given list final author, only keep in list the author that appear in final author list

lauthor_filtered = []
loid_filtered = []
lid_filtered = []
lnrv_filtered = []
ltext_filtered = []
lrate_filtered = []

for i in tqdm(range(len(lauthor))):
  if lauthor[i] in finalAuth:
    lauthor_filtered.append(lauthor[i])
    loid_filtered.append(loid[i])
    lid_filtered.append(lid[i])
    lnrv_filtered.append(lnrv[i])
    ltext_filtered.append(ltext[i])
    lrate_filtered.append(lrate[i])
      

print(len(lauthor_filtered), len(loid_filtered), len(lid_filtered), len(lnrv_filtered))


100%|████████████████████████████████████████████████████████████████████████| 878561/878561 [03:23<00:00, 4321.65it/s]

84904 84904 84904 84904


## 11974 user has more than 4 review

In [ ]:
df = pd.DataFrame({
    'user_id': lauthor_filtered,
    'rest_id': loid_filtered,
    'review_id': lid_filtered,
    'rating': lrate_filtered,
    'text': ltext_filtered
})
df.to_csv('./data/reviews/tripAdvisor.csv', index=False)

In [20]:
import random

test_percentage = 0.2
num_test_authors = int(len(finalAuth) * test_percentage)
test_authors = random.sample(finalAuth, num_test_authors)
train_authors = [author for author in finalAuth if author not in test_authors]


In [21]:
print("Number of authors in training set:", len(train_authors))
print("Number of authors in testing set:", len(test_authors))

Number of authors in training set: 9579
Number of authors in testing set: 2394


In [ ]:
#  save train and test to json as following: {"tripadvisor": {"train": ["PVyZXgOkVtnU6966FDFhuw",..], "test": ["PVyZXgOkVtnU6966FDFhuw",..]}}

data = {
    "tripAdvisor": {
        "train": train_authors,
        "test": test_authors
    }
}

with open('./data/reviews/tripAdvisor_splits.json', 'w') as f:
  json.dump(data, f)


## Hotel Information

In [17]:
def process_line2(line):
    try:
        data = json.loads(line.strip())
        id = data.get('id', '')
        locality = data.get('address', {}).get('locality', '')
        name = data.get('name', '')
        region = data.get('address', {}).get('region', '')
        region_id = data.get('region_id', '')

    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
    return id, locality, name, region, region_id

In [18]:
file_path = './data/reviews/hotel.txt'

counter = 0
hotelID = []
llocality = []
lname = []
lregion = []
lregion_id = []
with open(file_path, 'r', encoding='utf-8') as file:
  for line in file:
    id, locality, name, region, region_id = process_line2(line)
    hotelID.append(id)
    llocality.append(locality)
    lname.append(name)
    lregion.append(region)
    lregion_id.append(region_id)
    counter += 1
print(counter)

4333


In [19]:
df2 = pd.DataFrame({
    'hotelID': hotelID,
    'locality': llocality,
    'name': lname,
    'region': lregion,
    'region_id': lregion_id
})
df2.to_csv('./data/reviews/hotel.csv', index=False)

In [23]:
# given authorID, obtain number of region by hotelID

def get_region_by_author(authorID):
  df_filtered = df[df['user_id'] == authorID]
  hotelIDs = df_filtered['rest_id'].tolist()
  df2_filtered = df2[df2['hotelID'].isin(hotelIDs)]
  regions = df2_filtered['region_id'].unique()
  return regions

author_region ={}
for idx in tqdm(range(len(finalAuth))):
    authorID = finalAuth[idx]
    regions = get_num_region_by_author(authorID)
    author_region[authorID] = regions
#   print(f"Number of regions visited by author {authorID}: {num_regions}")

freq = Counter(author_freq)
print(freq)

100%|███████████████████████████████████████████████████████████████████████████| 11973/11973 [01:37<00:00, 122.23it/s]

Counter({3: 3198, 4: 3012, 2: 2043, 5: 1735, 1: 765, 6: 699, 7: 286, 8: 120, 9: 62, 10: 27, 11: 15, 12: 6, 14: 2, 13: 2, 16: 1})


In [27]:
for key in author_region:
  author_region[key] = author_region[key].tolist()
with open('./data/reviews/user2region.json', 'w') as f:
  json.dump(author_region, f)